$\newcommand{\calf}{{\cal F}}
\newcommand{\dnu}{d \nu}
\newcommand{\mf}{{\bf F}}
\newcommand{\vu}{{\bf u}}
\newcommand{\vx}{{\bf x}}
\newcommand{\vy}{{\bf y}}
\newcommand{\vs}{{\bf s}}
\newcommand{\ve}{{\bf e}}
\newcommand{\vd}{{\bf d}}
\newcommand{\mg}{{\bf G}}
\newcommand{\ml}{{\bf L}}
\newcommand{\mg}{{\bf G}}
\newcommand{\ma}{{\bf A}}
\newcommand{\mi}{{\bf I}}
\newcommand{\mm}{{\bf M}}
\newcommand{\ball}{{\cal B}}
\newcommand{\ptc}{{\Psi TC}}
\newcommand{\diag}{\mbox{diag}}
\newcommand{\begeq}{{\begin{equation}}}
\newcommand{\endeq}{{\end{equation}}}
$

In [ ]:
using SIAMFANLEquations
using SIAMFANLEquations.TestProblems
using PyPlot
using NotebookSIAMFANL

# Chapter 1: Introduction

## Section 1.1: What is the problem?

Nonlinear equations are solved as part of almost all simulations of
physical processes. Physical models that are expressed as
nonlinear partial differential equations, for example, become
large systems of nonlinear equations when discretized. Authors of
simulation codes must either use a nonlinear solver as a tool or
write one from scratch. The purpose of this book is to show
these authors what technology is available, sketch the implementation,
and warn of the problems. We do this via algorithmic outlines,
nonlinear solvers in Julia that can be used for
production work, a suite of example problems, an IJulia notebook,
and chapter-ending projects.

We use the standard notation 

\begin{equation}
\mf(\vx) = 0
\end{equation}

for systems of $N$ equations in $N$ unknowns. We will refer to this as the
__nonlinear equations formulation__.
Here $\mf:R^N  \to R^N$. We will call $\mf$ the
__nonlinear residual__
 or simply the __residual__.
Rarely can the solution of a nonlinear equation be given by
a closed-form expression, so iterative methods must be used
to approximate the solution numerically.
The output of an iterative method
is a sequence of approximations to a solution.

The __fixed-point__ formulation of a nonlinear equation is

$$
\vx = \mg(\vx) .
$$

The difference between the two formulations is not simply replacing
$\mf(\vx)$ by $\vx - \mg(\vx)$. The algorithms for $\mf(\vx) = 0$ take
a very different approach from those for fixed-point prolbems.

We will spend
most of our time in this introductory section on methods for the nonlinear equations formulation.
The reason for this is that much of the theory can be explored in the
simple context of scalar equations. We will consider fixed-point problems
seriously in [Chapter 4](SIAMFANLCh4.ipynb).



### Section 1.1.1: Notation


In this book, following the convention in 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>
and
<cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite>,
vectors are to be understood as column vectors.
Following
<cite data-cite="ctk:acta"><a href="siamfa.html#ctk:acta">(Kel18)</cite>
denote vectors by boldfaced lower case letters
and matrices by boldfaced upper case letters,
for example $\vx$  and $\ma$. We denote the $i$th component of $\vx$
by $x_i$ to distinguish between the $i$th member of a sequence of
vectors $\vx_i$. We denote the $ij$ entry of $\ma$ by $\ma_{ij}$.

The vector
$\vx^*$ will denote a solution, $x$ a potential solution,
and $\{ \vx_n \}_{n \ge 0}$ the sequence of iterates. We will refer to
$\vx_0$ as the
\index{Initial!guess}
\index{Initial!iterate}{\bf initial iterate (not guess!)}.
We will denote
the $i$th component of a vector $\vx$ by $x_i$
and the $i$th component of an index vector
$\vx_n$ by $x_{ni}$. We will rarely need
to refer to individual components of vectors.
We will let
$\partial \mf/\partial x_i$ denote the partial derivative of $\mf$
with respect to
$x_i$. As is standard
<cite data-cite="dens"><a href="siamfa.html#dens">(DS96)</cite>,
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
$\ve = \vx - \vx^*$ will denote the error. So, for example,
$\ve_n = \vx_n - \vx^*$ is the error in the $n$th iterate.
    
If the components of $\mf$ are differentiable at $\vx \in R^N$,
we define the __Jacobian matrix__
$\mf'(\vx)$ by
    
$$
\mf'(\vx)_{ij} = \frac{\partial f_i}{\partial x_j} (\vx).
$$
Throughout the book, $\| \cdot \|$ will denote the Euclidean norm
on $R^N$:
    
$$
\| \vx \| = \left( \sum_{i=1}^N x_i^2 \right)^{1/2}.
$$


We treat scalar equations with lowercase letters. So a scalar equation
is $f(x) = 0$ and the derivative is $f'(x)$.
Many of the essential ideas in this book can be illustrated with scalar
equations and we do that in this chapter. The exception is the need to
solver linear systems of equations and linear least squares problems, which
will be the focus of the remaining chapters in the book.
The Julia codes for the examples in this section are in the
[src/Chapter1](src/Chapter1)
directory for the notebook. The solvers and
test problems are part of the [SIAMFANLEquations](https://github.com/ctkelley/NotebookSIAMFANL) Julia package.
    


## Section 1.2: Newton's Method

Most of the methods in this book are variations of Newton's method.
The exception will be Anderson acceleration, a solver for fixed-point problems, which we cover in
Chapter~\ref{ch:anderson}.

The Newton sequence is

$$
\vx_{n+1} = \vx_n - \mf'(\vx_n)^{-1} \mf(\vx_n).
$$

The interpretation of \eqnok{newtonseq} is that
we model $\mf$ at the current iterate $\vx_n$ with a linear
function
$$
\mm_n(x) = \mf(\vx_n) + \mf'(\vx_n) (\vx - \vx_n)
$$
and let the root of $\mm_n$ be the next iteration.
$\mm_n$ is called the
__local linear model__.
If $\mf'(\vx_n)$ is
nonsingular, then the Newton sequence is the solution of $\mm_n(\vx_{n+1}) = 0$.

We illusrate  the local linear model and the
Newton iteration for the scalar equation
\[
\arctan(x) = 0
\]
with initial iterate $x_0 = 1$. We graph the local linear model
\[
m_j(x) = f(x_j) + f'(x_j) (x - x_j)
\]
at $x_j$ from the point $(x_j, y_j) = (x_j, f(x_j))$ to the next iteration
$(x_{j+1},0)$. The iteration converges rapidly and one can see the
linear model becoming more and more accurate. The third iterate is
visually indistinguishable from the solution.

Run the code window below to see the plot. The code __atan_test.jl__ runs the solver
__nsolsc.jl__ from the[SIAMFANLEquations](https://github.com/ctkelley/NotebookSIAMFANL) Julia package,
collects the data for the plot, and then runs some messy PyPlot commands. We will discuss the solver in detail
in [Section 1.10](#Section-1.10:-Scalar-Equation-Solver). For now we will focus on the results

The Julia program __fig1dot1.jl__
creates Figure 1.1 in this chapter. The semicolon after the function call suppresses some unnecessary output from PyPlot. Remove to see what happens.

In [ ]:
fig1dot1();

The computation of a Newton iteration requires

1. evaluation of $\mf(\vx_n)$ and a test for termination,
2. approximate solution of the equation
$\mf'(\vx_n) \vs = - \mf(\vx_n)$
for the Newton step $\vs$, and
3. construction of $\vx_{n+1} = \vx_n + \lambda \vs$, where the
step length $\lambda$ is selected to guarantee decrease in $\| \mf \|$.


The computation of the Newton step,
consumes most of the work, and the variations
in Newton's method that we discuss in this book differ most significantly
in how the Newton step is approximated. Computing the
step may require evaluation and factorization of the Jacobian matrix
or the solution of the linear equation by an iterative method.
Not all methods for computing
the Newton step require the complete Jacobian matrix, which, as we
will see in [Chapter 2](SIAMFANLCh2.ipynb)
can be very expensive.

In the example from Figure 1.1, the step $s$
in  was
satisfactory, and we can use $\lambda=1$ in step 3. The reader
should be warned that attention to the step length is generally very
important. One should not write one's own nonlinear solver without
step-length control (see [Section 1.6](#Section-1.10:-Global_Convergence_and_the_Armijo_Rule)).



### Section 1.2.1: Local Convergence Theory

The convergence theory for Newton's method 
<cite data-cite="dens"><a href="siamfa.html#dens">(DS96)</cite>,
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
<cite data-cite="ortega"><a href="siamfa.html#ortega">(OR70)</cite>,
that is most often seen in an elementary course in numerical methods
is __local__. This means
that one assumes that the __initial iterate__ $\vx_0$ is near a solution.
The local convergence
theory from
<cite data-cite="dens"><a href="siamfa.html#dens">(DS96)</cite>,
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
<cite data-cite="ortega"><a href="siamfa.html#ortega">(OR70)</cite>,
requires the __standard assumptions__.
    
---
__Assumption:__

1. The equation $\mf(\vx) = 0$ has a solution $\vx^*$.
2. $\mf': \Omega \to R^{N \times N}$ is Lipschitz continuous near $\vx^*$.}
3. $\mf'(\vx^*)$ is nonsingular.
---

Recall that Lipschitz continuity near $\vx^*$
means that there is $\gamma > 0$
(the __Lipschitz constant__) such that
    
$$
\| \mf'(\vx) - \mf'(\vy) \| \le \gamma \| \vx - \vy \|
$$
for all $\vx, \vy$ sufficiently near $\vx^*$.

We state the classic local convergence theorem.

___
__Theorem 1.1__ Let the standard assumptions hold. If $\vx_0$ is sufficiently
near $\vx^*$, then the Newton sequence exists
(i.e., $\mf'(\vx_n)$ is nonsingular
for all $n \ge 0$) and converges to $\vx^*$ and there is $K > 0$ such that
    
$$
\| \ve_{n+1} \| \le K \| \ve_n \|^2
$$
for $n$ sufficiently large.
___


The convergence described by the theorem, in which
the error in the solution will be roughly squared with each iteration,
is called  __q-quadratic__. 
Squaring the error roughly means that the number of significant
figures in the result doubles with each iteration.  Of course,
one cannot examine the error without knowing the solution. However,
we can observe the quadratic reduction in the error computationally,
if $\mf'(\vx*)$ is well conditioned,
because the nonlinear residual
will also be roughly squared with each iteration.
Therefore, we should see the
exponent field of the norm of the nonlinear residual roughly double
with each iteration.
    
In Table 1.1 we report the Newton iteration
for the scalar ($N=1$) nonlinear equation
    
$$
f(x) = \tan(x) - x = 0, \, x_0 = 4.5.
$$
The solution is $x^* \approx 4.493$.

The decrease in the function
is as the theory predicts for the first three iterations, then progress
slows down for iteration 4 and stops completely after that. The
reason for this __stagnation__
is clear: one cannot evaluate the function
to higher precision than (roughly) machine unit roundoff, which in
the IEEE
<cite data-cite="ieee"><a href="siamfa.html#ieee">(IEE85)</cite>,
<cite data-cite="IEEEnew"><a href="siamfa.html#IEEEnew">(IEE19)</cite>,
<cite data-cite="overtonbook"><a href="siamfa.html#overtonbook">(Ove01)</cite>
floating point system is about $10^{-16}$.

We make Table 1.1 with __tab1dot1.jl__.


In [ ]:
tab1dot1()

Stagnation is not affected by the accuracy in the derivative. The
results reported in Table 1.1 used a
forward difference approximation to the derivative with a difference
increment of $10^{-6}$. With this choice of difference increment, the
convergence speed of the nonlinear iteration is as fast as
that for Newton's method, at least for this example, until stagnation
takes over.
The reader should be aware that difference approximations to derivatives,
while usually reliable, are often expensive and can be very inaccurate.
An inaccurate Jacobian can
cause many problems (see 
[Section 1.9](#Section-1.9:-What_Can_Go_Wrong) )

An analytic Jacobian
can require some human effort, but can be worth it in terms of
computer time and robustness when a difference Jacobian performs poorly.

One can quantify this stagnation by adding the errors in the function
evaluation and derivative evaluations to __Theorem 1.1__
The messages of __Theorem 1.2__ are:

---
1. Small errors, for example, machine roundoff, in the function evaluation can lead to stagnation. This type of stagnation is usually benign and, if the Jacobian is well conditioned (see the estimate in [Section 1.5](#Section1.5:-Termination_of_the_Iteration) ),the results will be as accurate as the evaluation of $\mf$

2. Errors in the Jacobian and in the solution of the linear equation for the Newton step \eqnok{nstep} will affect the speed of the nonlinear iteration, but not the limit of the sequence.

---



__Theorem 1.2__ 
Let the standard assumptions hold. Let a matrix-valued
function $\Delta(\vx)$ and a vector-valued function $\epsilon(\vx)$
be such that
$$
\| \Delta(\vx) \| < \delta_J \mbox{ and }
\| \epsilon(\vx) \| < \delta_F
$$
for all $\vx$ near $\vx^*$.
Then, if $\vx_0$ is sufficiently near $\vx^*$ and $\delta_J$ and
$\delta_F$ are sufficiently small, the sequence
$$
\vx_{n+1} = \vx_n -
( \mf'(\vx_n) + \Delta(\vx_n))^{-1} (\mf(\vx_n) + \epsilon(\vx_n))
$$
is defined (i.e., $\mf'(\vx_n) + \Delta(\vx_n)$ is nonsingular for all $n$)
and satisfies
$$
\| \ve_{n+1} \| \le {\bar K} ( \| \ve_n \|^2 + \| \Delta(\vx_n) \|
\| \ve_n \|
+ \| \epsilon(\vx_n)\| )
$$
for some ${\bar K} > 0$.

***

We will ignore the errors in the function in the rest of this book,
but one needs to be aware that stagnation of the nonlinear iteration
is all but certain in finite-precision arithmetic. However, the
asymptotic convergence results for exact arithmetic
describe the observations well for most problems.

While Table 1.1
gives a clear picture of quadratic convergence, it's
easier to appreciate a graph.
Figure 1.2 is a semilog plot of
__residual history}__ i.e.,
the norm of the nonlinear residual against the iteration number.
The concavity of the plot is the signature of superlinear convergence.
One uses the __semilogy__ command from the __PyPlot__ package
in Julia for this. See the
file __fig1dot2.jl__, which generated Figure 1.2
for an example.



In [ ]:
fig1dot2();

## Section 1.3: Approximating the Jacobian

As we will see in the subsequent chapters, it is usually most efficient
to approximate the Newton step in some way.
One way to do this is
to approximate $\mf'(\vx_n)$ in a way that not only avoids computation
of the derivative, but also saves linear algebra work and matrix storage.

The price for such an approximation is that the nonlinear iteration
converges more slowly; i.e., more nonlinear iterations
are needed to solve the problem.
However, the overall cost of the solve
is usually significantly less, because the computation of the
Newton step is less expensive.

One way to approximate the Jacobian is to
compute $\mf'(\vx_0)$ and use that as an approximation to
$\mf'(\vx_n)$
throughout the iteration.
This is the __chord method__
or __modified Newton method__.
The convergence
of the chord iteration is not as fast as Newton's method. Assuming
that the initial iteration is near enough to $x^*$, the convergence
is __q-linear__.
This means that there is $\rho \in (0,1)$ such that
$$
\| \ve_{n+1} \| \le \rho \| \ve_{n} \|
$$
for $n$ sufficiently large.
We can apply __Theorem: (Approximate)__ to the chord method with
$\epsilon=0$ and
$\| \Delta(\vx_n) \| = O(\| \ve_0 \|)$ and conclude that $\rho$ is
proportional to the initial error.
The constant $\rho$ is called the  __q-factor__.
The formal definition of q-linear convergence allows for faster
convergence.
Q-quadratic convergence is also q-linear, as you can see from the
definition.
In many cases of q-linear convergence, one observes that
$$
\| \ve_{n+1} \| \approx \rho \| \ve_{n} \|
\mbox{ or }
\| \mf(\vx_{n+1}) \| \approx \rho \| \mf(\vx_{n}) \|.
$$
In these cases,
q-linear convergence is usually easy to see on a semilog plot of the
residual norms against the iteration number. The curve appears to
be a line with slope $\approx \log(\rho)$.

The __secant method__ 
for scalar equations approximates the
derivative using a finite difference, but, rather than a forward
difference, uses the most recent two iterations to form
the difference quotient. So
$$
x_{n+1} = x_n - \frac{f(x_n)(x_n - x_{n-1})}{f(x_n)- f(x_{n-1})},
$$
where $x_n$ is the current iteration and $x_{n-1}$ is the
iteration before that. The secant method must be initialized
with two points. One way to do that is to let
$x_{-1} = 0.99 x_0$. This is what we do in our Julia code __nsolsc.jl__, which
includes the Newton, chord, and secant methods as options.

The formula for the secant method does not extend to
systems of equations ($N > 1$) because the denominator in the fraction
would be a difference of vectors. We discuss one of the
many generalizations of the secant method for systems of equations
in 
[Chapter 5](SIAMFANLCh5.ipynb).

The secant method's approximation to $f'(x_n)$ converges to $f'(x^*)$
as the iteration progresses. __Theorem 1.2__, with
$\epsilon=0$ and $\| \Delta(x_n) \| = O(\| e_{n-1} \|)$, implies that
the iteration
converges __q-superlinearly__. This means that either $x_n = x^*$
for some finite $n$ or
$$
\lim_{n \to \infty} \dfrac{\| e_{n+1} \|}{\| e_{n} \|} = 0.
$$
Q-superlinear convergence is hard to distinguish from q-quadratic
convergence by visual inspection of the semilog plot of the residual history.
The residual curve for q-superlinear convergence is concave down
but drops less rapidly than the one for Newton's method.



## Section 1.4: Inexact Newton Methods

Rather than approximate the Jacobian, one could instead solve
the equation for the Newton step approximately. An
__inexact Newton method__
<cite data-cite="demboes"><a href="siamfa.html#demboes">(Des82)</cite>
uses as a Newton step
a vector $\vs$ that satisfies the
__inexact Newton condition__

$$
\| \mf'(\vx_n) \vs + \mf(\vx_n) \| \le \eta \| \mf(\vx_n) \|.
$$
    
The parameter $\eta$ (the __forcing term__)
can be varied as the Newton iteration progresses.
Choosing a small value of $\eta$ will make the iteration more like
Newton's method, therefore leading to convergence in fewer iterations.
However, a small value of $\eta$ may make computing a step
that satisfies \eqnok{inexact} very expensive. The local convergence
theory
<cite data-cite="demboes"><a href="siamfa.html#demboes">(Des82)</cite>,
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>
for inexact Newton methods reflects the intuitive idea that a small
value of $\eta$ leads to fewer iterations. __Theorem 1.3__
is a typical example of such a convergence result.
    
    
---
__Theorem 1.3__
Let the standard assumptions hold. Then there are $\delta$ and $\bar \eta$
such that, if $\| \ve_0 \| \le \delta$ and
$\{ \eta_n \} \subset [0, \bar \eta]$,
then the inexact Newton iteration
\[
\vx_{n+1} = \vx_n + \vs_n,
\]
where
$$
\| \mf'(\vx_n) \vs_n + \mf(\vx_n) \| \le \eta_n \| \mf(\vx_n) \|,
$$
converges q-linearly to $\vx^*$. Moreover,

- if $\eta_n \to 0$, the convergence is q-superlinear, and
- if $\eta_n \le K_\eta \| \mf(\vx_n) \|^p$ for some
$K_\eta > 0$, the convergence is q-superlinear with q-order $1+p$.
    
___

Errors in the function evaluation will, in general, lead to
stagnation of the iteration.

One can use __Theorem 1.3__ to analyze the chord method or
the secant method. In the case of the chord method, the steps
satisfy the inexact Newton condition with
$$
\eta_n = O(\| \ve_0 \|),
$$
which implies q-linear convergence if $\| \ve_0 \|$ is sufficiently small.
For the secant method, $\eta_n = O(\| \ve_{n-1} \|)$, implying
q-superlinear convergence. 
    
Iterative methods (such as GMRES
<cite data-cite="gmres"><a href="siamfa.html#gmres">(SS86)</cite>) for solving
the equation for the Newton step would typically use
the inexact Newton condition as a termination criterion. In this case, the
overall nonlinear solver is called a
__Newton iterative method__.
Newton iterative methods are named by the
particular iterative method used for the linear equation. For
example, the __nsoli.jl__ code,
which we describe in [Chapter 3](SIAMFANLCh3.ipynb), is an implementation
of some __Newton-Krylov__ methods.

An unfortunate choice of the forcing term $\eta$ can lead to
very poor results. The reader is invited to try the two
choices $\eta = 10^{-6}$ and $\eta = .9$ in __nsoli.jl__
to see this. Better choices of $\eta$ include $\eta = 0.1$,
the author's personal favorite, and a more complex approach
(see [Chapter 3](SIAMFANLCh3.ipynb))
from 
<cite data-cite="homerstan"><a href="siamfa.html#homerstan">(EW94)</cite>
and
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>
that is the
default in __nsoli.jl__. Either of these  usually leads to
rapid convergence near the solution, but at a much lower cost
for the linear solver than a very small forcing term such as
$\eta = 10^{-4}$.



## Section 1.5: Termination of the Iteration

While one cannot know the error without
knowing the solution, in most cases the norm of $\mf(\vx)$ can be used
as a reliable indicator of the rate of decay in $\| \ve \|$ as the
iteration progresses \cite{ctk:roots}. Based on this heuristic, we
terminate the iteration in our codes when

$$
\| \mf(\vx) \| \le \tau_r \| \mf(\vx_0) \| + \tau_a.
$$

The relative $\tau_r$
and absolute $\tau_a$ error tolerances are
both important. Using only the relative reduction in the
nonlinear residual as a basis for termination (i.e., setting $\tau_a = 0$)
is a poor idea because an initial iterate that is near the solution
may make the criterion impossible to satisfy with $\tau_a = 0$.

One way to quantify the utility of termination when
$\| \mf(\vx) \|$
is small is to compare a relative reduction in the norm of the
error with a relative reduction in the norm of the nonlinear residual.
If the standard assumptions hold and
$x_0$ and $x$ are sufficiently near the root, then
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>

$$
\frac{\| \ve \|}{4 \| \ve_0 \| \kappa(\mf'(\vx^*))} \le
\frac{\| \mf(\vx) \|}{\| \mf(\vx_0) \|}
\le \frac{4 \kappa(\mf'(x^*)) \| \ve \|}{\| \ve_0 \|},
$$
    
where
    
$$
\kappa(\mf'(\vx^*)) = \| \mf'(\vx^*)\| \| \mf'(\vx^*)^{-1} \|
$$
    
is the
__condition number__ of $\mf'(\vx^*)$ relative to the norm
$\| \cdot \|$. From the estimate above which compares relative error and
relative resiual, we conclude that, if the
Jacobian is well conditioned (i.e., $\kappa(\mf'(\vx^*))$ is not
very large), then our termination criterion useful.
This is analogous to the linear case, where
a small residual implies a small error if the matrix is well conditioned.
    
Another approach, which is supported by theory only for superlinearly
convergent methods,
is to exploit the fast convergence to estimate the error in
terms of the step. If the iteration is converging superlinearly, then
    
$$
\ve_{n+1} = \ve_n + \vs_n = o(\| \ve_n \|)
$$
    
and hence
    
$$
\vs_n = -\ve_n + o(\| \ve_n \|).
$$
    
Therefore, when the iteration is converging superlinearly, one may
use $\| \vs_n \|$ as an estimate of $\| \ve_n \|$. One can estimate the
current rate of convergence from above by
    
$$
\rho_n = \| \vs_n \|/\| \vs_{n-1} \| \approx \| \ve_n \|/\| \ve_{n-1} \|
\ge \| \ve_{n+1} \|/\| \ve_n \|.
$$
    
Hence, for $n$ sufficiently large,
    
$$
\| \ve_{n+1} \| \le \rho_n \| \ve_n \| \approx \| \vs_n \|^2/\| \vs_{n-1}\|.
$$
    
So, for a superlinearly convergent method, terminating the iteration
with $\vx_{n+1}$ as soon as

$$
\| \vs_n \|^2/\|\vs_{n-1}\| < \tau
$$
will imply that $\| \ve_{n+1} \| < \tau$. This is __termination on small steps__.
    
Termination on small steps
is only supported by theory for superlinearly convergent
methods, but is used for linearly convergent methods in some
initial value problem solvers 
<cite data-cite="slc"><a href="siamfa.html#slc">(BCP96)</cite>,
<cite data-cite="lsode"><a href="siamfa.html#lsode">(RH93)</cite>.
The trick is
to estimate the q-factor $\rho$, say, by
    
$$
\rho \approx \| \vs_n \|/\| \vs_{n-1} \|
\mbox{ or }
\rho \approx (\| \vs_n \|/\| \vs_0 \|)^{1/n}.
$$
    
Assuming that the estimate of $\rho$ is reasonable, then
    
$$
\| \ve_n \| - \| \vs_n \| \le \| \ve_{n+1} \| \approx \rho \| \ve_n \|
$$
    
implies that
    
$$
\| \ve_{n+1} \|/\rho \approx \| \ve_n \| \le \| \vs_n \|/(1 - \rho).
$$
    
Hence, if we terminate the iteration when

$$
\| \vs_n \| \le \tau (1 - \rho)/\rho
$$
    
and the estimate of $\rho$ is an __overestimate__, then
the termination on small steps criterion will imply that
    
$$
\| \ve_{n+1} \| \le \rho \| \vs_n \|/(1-\rho)  \le \tau.
$$
    
In practice, a safety factor is used on the left side of these criteria
to guard against an underestimate.

If, however, the estimate of $\rho$ is much smaller than the actual
q-factor, the iteration can terminate too soon. This can happen
in practice if the Jacobian is ill conditioned and the initial iterate
is far from the solution
<cite data-cite="ctk:mike2"><a href="siamfa.html#ctk:mike2">(KMT98)</cite>.


## Section 1.6: Global Convergence and the Armijo Rule

The requirement in the local convergence theory
that the initial iterate be near the solution is
more than mathematical pedantry. To see this, we apply
Newton's method to find the root $x^* = 0$ of the
function $f(x) = \arctan(x)$ with initial iterate $x_0 = 10$. This
initial iterate is too far from the root for the local convergence
theory to hold. In fact, the step

$$
s = \frac{f(x_0)}{f'(x_0)} \approx \frac{1.5}{-0.01} \approx -150,
$$

while in the correct direction, is far too large in magnitude.

The initial iterate and the four subsequent iterates are

$$
10, -138, 2.9 \times 10^4, -1.5 \times 10^9, 9.9 \times 10^{17}.
$$

As you can see, the Newton step points in the correct direction,
i.e., toward $x^* = 0$, but overshoots by larger and larger amounts.
The simple artifice of reducing the step by half
until $\|\mf(\vx)\|$ has been reduced will usually solve this problem.

In order to clearly describe this, we will now
make a distinction between the
__Newton direction__
$\vd = -\mf'(x)^{-1} \mf(x)$ and the
__Newton step__ when we discuss global
convergence. For the methods in this book, the Newton step
will be a positive scalar multiple of the Newton direction.
When we talk about local convergence and are taking
full steps ($\lambda = 1$ and $\vs = \vd$), we will not make this
distinction and only refer to the step, as we have been doing
up to now.

A rigorous convergence analysis requires a bit more detail. We
begin by computing the __Newton direction__
$$
\vd = - \mf'(\vx_n)^{-1} \mf(\vx_n).
$$

To keep the step from going too far, we find the smallest integer $m \ge 0$
such that the __sufficient decrease condition__

$$
\|\mf(\vx_n + 2^{-m} \vd) \| < (1 - \alpha 2^{-m} ) \| \mf(\vx_n) \|
$$

holds. We let the step be $\vs = 2^{-m} \vd$ and
$\vx_{n+1} = \vx_n + 2^{-m} \vd$.

The parameter $\alpha \in (0,1)$ is a small
number intended to make the sufficient decrease condition as easy as possible
to satisfy.
$\alpha = 10^{-4}$ is typical and used in our codes.

In the figure (Figure 1.4 in the print book) 
created by __fig1dot4.jl__,
we show how this approach, called the
__Armijo rule__ 
<cite data-cite="armijo"><a href="siamfa.html#armijo">(Arm66)</cite>,
succeeds. The
circled points are iterations for which $m > 1$ and the value of
$m$ is above the circle.



In [ ]:
fig1dot4();

Methods like the Armijo rule are called
__line search__ methods because one searches
for a decrease in $\| \mf \|$ along the line segment
$[\vx_n, \vx_n + \vd]$.

The line search in our codes manages the reduction in the step size
with more sophistication than simply halving an unsuccessful step.
The motivation
for this is that some problems respond well to one or two reductions
in the step length by modest amounts (such as 1/2)
and others require many such reductions, but might do much
better if a more
aggressive step-length reduction (by factors of 1/10, say) is used.
To address this possibility, after two reductions by halving do
not lead to sufficient decrease, we build a quadratic polynomial model of
$$
\phi(\lambda) = \|F(\vx_n + \lambda \vd) \|^2
$$
based on interpolation of $\phi$ at the three most
recent values of $\lambda$. The next $\lambda$ is the
minimizer of the quadratic model, subject to the
__safeguard__ that the reduction in $\lambda$ be at least
a factor of two and at most a factor of ten. So
the algorithm generates a sequence of candidate step-length factors
$\{ \lambda_m \}$ with $\lambda_0 = 1$ and
$$
1/10 \le \lambda_{m+1}/\lambda_m \le 1/2.
$$
The norm in this equation is, at least for theory, understood to be the
$\ell^2$ norm and is squared to make $\phi$ a smooth
function that can be accurately modeled by a quadratic over
small ranges of $\lambda$.

In the advanced codes from the subsequent chapters,
we use the three-point parabolic model from 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>. In
this approach, $\lambda_1 = 1/2$. To compute $\lambda_m$ for
$m > 1$, a parabola is fitted to the data $\phi(0)$,
$\phi(\lambda_m)$, and
$\phi(\lambda_{m-1})$. $\lambda_m$ is the minimum of this parabola on the
interval $[\lambda_{m-1}/10,\lambda_{m-1}/2]$. We refer the reader
to 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite> for the details and
to 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
<cite data-cite="ortega"><a href="siamfa.html#ortega">(OR70)</cite>,
<cite data-cite="dens"><a href="siamfa.html#dens">(DS96)</cite>,
<cite data-cite="homerstan"><a href="siamfa.html#homerstan">(EW94)</cite>
for a discussion of other ways
to implement a line search.

In Figure 1.5 we apply the 
parabolic line search to the problem from Figure 1.4.
As you can see iteration with the polynomial line search avoids the 
repeated stepsize reductions early in the iteration and finds the 
solution in significantly fewer function evaluations.

In [ ]:
fig1dot5();

### Section 1.6.1:  A Basic Algorithm

Algorithm __nsolg__ is a general formulation of an inexact
Newton--Armijo iteration. The methods in 
[Chapter 2](SIAMFANLCh2.ipynb) and
[Chapter 3](SIAMFANLCh3.ipynb)
are special cases of __nsolg__. There is
a lot of freedom in Algorithm __nsolg__. The essential input
arguments are the initial iterate $\vx$, the function $\mf$, and
the relative and absolute termination tolerances $\tau_a$
and $\tau_r$. If __nsolg__ terminates successfully,
$\vx$ will be the approximate solution on output.

Within the algorithm, the computation of the
Newton direction $\vd$ can be done with direct or iterative
linear solvers, using either the Jacobian $\mf'(\vx)$ or an
approximation of it. If you use a direct solver, then
the forcing term $\eta$ is determined implicitly; you do
not need to provide one. For example, if you solve the
equation for the Newton step with a direct method, then
$\eta = 0$ in exact arithmetic. If you use an approximate
Jacobian and solve with a direct method, then $\eta$ is
proportional to the error in the Jacobian. Knowing about
$\eta$ helps you understand and apply the theory, but is not
necessary in practice if you use direct solvers.

If you use an iterative linear solver, then usually
the inexact Newton condition is the termination criterion for that
linear solver. You'll need to make a decision about the forcing term
in that case (or accept the defaults from a code like
__nsoli.jl__, which we describe in [Chapter 3](SIAMFANLCh3.ipynb).
The theoretical
requirements on the forcing term $\eta$ are that it be safely
bounded away from one.

Having computed the Newton direction, we compute a step length
$\lambda$ and a step $\vs = \lambda \vd$ so that the sufficient
decrease condition holds. It's standard in line search
implementations to use a polynomial model like the one we described
above.

The algorithm does not cover all aspects of a useful implementation.
The number of nonlinear iterations, linear iterations, and
changes in the step length all should be limited. Failure of any
of these loops to terminate reasonably rapidly
indicates that something is wrong. We list some of the potential
causes of failure in the subsequent sections.

---

![Alg1.1](Images/Alg1dot1.png)


The theory for Algorithm __nsolg__ is very satisfying.
If $\mf$ is sufficiently smooth, $\eta$
is safely bounded away from one,
the Jacobians remain well conditioned throughout the iteration, and
the sequence $\{ \vx_n \}$
remains bounded, then the iteration converges to a solution
and, when near the solution, the convergence is as fast as the quality of
the linear solver permits. T__heorem 1.4__ states this precisely,
but not as generally as the results in 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
<cite data-cite="dens"><a href="siamfa.html#dens">(DS96)</cite>,
<cite data-cite="ortega"><a href="siamfa.html#ortega">(OR70)</cite>.
The important thing that you should remember is that, for smooth $F$,
there are only three possibilities for the iteration of
Algorithm __nsolg__:

- $\{ \vx_n \}$ will converge to a solution $x^*$, at which the standard
assumptions hold,
- $\{ \vx_n \}$ will be unbounded, or
- $\mf'(\vx_n)$ will become singular.


While the line search paradigm is the simplest way to find a solution
if the initial iterate is far from a root, other methods are available
and can sometimes overcome stagnation or, in the case of
many solutions, find the solution that
is appropriate to a physical problem.

__Trust region__ globalization 
<cite data-cite="dens"><a href="siamfa.html#dens">(DS96)</cite>,
<cite data-cite="powelltreq"><a href="siamfa.html#powelltreq">(Pow70)</cite>
__pseudotransient continuation__ 
<cite data-cite="ctk:pst"><a href="siamfa.html#ctk:pst">(KK98)</cite>,
<cite data-cite="ctk:ptc2"><a href="siamfa.html#ctk:ptc2">(CKK03)</cite>,
<cite data-cite="highamptc"><a href="siamfa.html#highamptc">(Hig99)</cite>,
<cite data-cite="deufptc"><a href="siamfa.html#deufptc">(Deu02)</cite>
and 
__homotopy__ methods 
<cite data-cite="bertini"><a href="siamfa.html#bertini">(BHSW13)</cite>,
<cite data-cite="hompack"><a href="siamfa.html#hompack">(WBM87)</cite>
are three such alternatives. 
We will cover pseudotransient continuation in some detail the next section.

---
    
__Theorem 1.4:__
Let $\vx_0 \in R^N$ and $\alpha \in (0,1)$ be given.
Assume that $\{ \vx_n \}$ is given by Algorithm __nsolg__,
$\mf$ is Lipschitz continuously differentiable,
    
$$
\{ \eta_n \} \subset (0, \bar \eta] \subset (0,1-\alpha),
$$
    
and $\{ \vx_n \}$ and $\{\| \mf'(\vx_n)^{-1} \|\}$ are bounded.
Then $\{ \vx_n \}$ converges to a root $\vx^*$ of
$\mf$ at which the standard assumptions hold, full steps
($\lambda = 1$) are taken for
$n$ sufficiently large, and the convergence behavior in the final phase
of the iteration is that given by the local theory for inexact
Newton methods (__Theorem 1.3__).
    
---


### Section 1.6.2: Warning!

The theory for global convergence of the inexact Newton--Armijo iteration
is only valid if $\mf'(\vx_n)$, or a very good approximation
(forward difference, for example), is used to compute the step.
A poor approximation to the Jacobian will cause the Newton step
to be inaccurate. While this can result in slow convergence when
the iterations are near the root, the outcome can be much worse
when far from a solution. The reason for this is that the success
of the line search is very sensitive to the direction.
In particular, if
$\vx_0$ is far from $\vx^*$ there is __no reason__ to expect the
secant or chord method to converge. Sometimes methods like
the secant and chord methods work fine with a line search when
the initial iterate is far from a solution, but users of
nonlinear solvers should be aware that the line search can fail.
A good code will watch for this failure and respond by
using a more accurate Jacobian or Jacobian-vector product.

Difference approximations to the Jacobian are usually sufficiently
accurate. However, there are particularly hard problems
<cite data-cite="kerksaad"><a href="siamfa.html#kerksaad">(KS92)</cite>
for which differentiation in the coordinate
directions is very inaccurate, whereas differentiation in
the directions of the iterations, residuals, and steps,
which are natural directions
for the problem, is very accurate.
The inexact Newton methods, such as the
Newton-Krylov methods in 
[Chapter 3](SIAMFANLCh3.ipynb), use a forward
difference approximation for Jacobian-vector products
(with vectors that are natural for the problem) and, therefore,
will usually (but not always) work well when far from a solution.


## Section 1.7: Pseudo-Transient Continuation

Nonlinear equations can have multiple solutions and even if the Newton-Armijo iteration converges to a solution, that may not be the solution you want. __Pseudo-Transient Continuation__ ($\ptc$) is one way to differentiate solutions you want from the ones you don't.

## Section 1.8: Things to Consider

## Section 1.9: What Can Go Wrong?


## Section 1.10: Scalar Equation Solver